In [177]:
import pandas as pd
import glob
from collections import defaultdict
import altair as alt
from pyvis.network import Network
import networkx as nx
import json


In [206]:
rend_individual = pd.read_excel("data procesada/resultados_tenis.xlsx")


In [178]:
# Lleva a una lista todos los nombres de los archivos necesarios
file_list = glob.glob('data/atp_matches_*.csv')

#### Diccionarios necesarios para los analisis de rendimientos, total de partidos, enfrentamientos directos, etc...

In [179]:
MostWinsDict = defaultdict(lambda: {'Victorias': 0, 'Total_Partidos': 0})
# victorias = defaultdict(int)
TotalPartidos = defaultdict(int)
H2HDict = defaultdict(lambda: {'total_matches': 0, 'wins_player1': 0, 'wins_player2': 0})

for file in file_list:
    df = pd.read_csv(file)
    for _, row in df.iterrows():
        winner = row['winner_name']
        loser = row['loser_name']

        MostWinsDict[winner]['Victorias'] += 1
        MostWinsDict[winner]['Total_Partidos'] += 1
        MostWinsDict[loser]['Total_Partidos'] += 1

        # victorias[winner] += 1
        TotalPartidos[winner] += 1
        TotalPartidos[loser] += 1

        H2HDict[(winner, loser)]['total_matches'] += 1
        H2HDict[(loser, winner)]['total_matches'] += 1

        H2HDict[(winner, loser)]['wins_player1'] += 1
        H2HDict[(loser, winner)]['wins_player2'] += 1

with open('data procesada/most_wins.json', 'w') as f:
    json.dump(dict(MostWinsDict), f)

with open('data procesada/total_partidos.json', 'w') as f:
    json.dump(dict(TotalPartidos), f)

H2HDict_str = {str(k): v for k, v in H2HDict.items()}

with open('data procesada/h2h.json', 'w') as f:
    json.dump(H2HDict_str, f)


### dataframe que me permite ver el rendimiento por año de los tenias

In [180]:
i = 1968
result = pd.DataFrame()

for file in file_list:
    df = pd.read_csv(file)

    winners = df.groupby("winner_name")["winner_name"].count().reset_index(name= "victorias")
    losers = df.groupby("loser_name")["loser_name"].count().reset_index(name= "derrotas")
    winners = winners.rename(columns={"winner_name": "jugador"})
    losers = losers.rename(columns={"loser_name": "jugador"})

    year_result = pd.merge(winners, losers, on="jugador", how="left")

    year_result["total partidos"] = year_result["victorias"] + year_result["derrotas"]
    year_result["Rendimiento"] = year_result["victorias"] * 100 / year_result["total partidos"]
    year_result["año"] = i  # Agrega una columna con el año

    result = pd.concat([result, year_result])  # Concatena el resultado del año actual con el resultado general

    i += 1

result.to_excel("data procesada/resultados_tenis.xlsx", index=False)


In [181]:
import pandas as pd
import altair as alt

def BuscarH2HJugador(diccionario, jugador):
    # Filtramos directamente con un diccionario de comprensión
    return {key: data for key, data in diccionario.items() if key[0] == jugador}

def GraficaH2H(nombre_jugador):
    H2Hjugador = BuscarH2HJugador(H2HDict, nombre_jugador)

    # Utilizamos lista de comprensión para crear las listas
    Rival = [y for (x, y), data in H2Hjugador.items() if data["total_matches"] >= 1]
    victorias = [data["wins_player1"] for (x, y), data in H2Hjugador.items() if data["total_matches"] >= 1]
    Derrotas = [data["total_matches"] - data["wins_player1"] for (x, y), data in H2Hjugador.items() if data["total_matches"] >= 1]

    data = pd.DataFrame({
        'Rival': Rival,
        'Victorias': victorias,
        'Derrotas': Derrotas
    })

    # Calcular total de enfrentamientos
    data['Total'] = data['Victorias'] + data['Derrotas']
    data = data.sort_values(by='Total', ascending=False).head(10)

    # Crear el gráfico de barras apiladas
    chart = alt.Chart(data).transform_fold(
        ['Victorias', 'Derrotas'],
        as_=['Resultado', 'Cantidad']
    ).mark_bar().encode(
        y=alt.Y('Rival:N', sort='-x'),
        x='Cantidad:Q',
        color='Resultado:N',
        tooltip=['Rival', 'Victorias', 'Derrotas', 'Total']
    ).properties(
        width=600,
        height=400,
        title='Head-to-Head de ' + nombre_jugador 
    )

    return chart

# Ejemplo de uso
chart = GraficaH2H('Novak Djokovic')
chart


alt.Chart(...)

In [182]:
df_most_wins = pd.DataFrame.from_dict(MostWinsDict, orient='index').reset_index()
df_most_wins = df_most_wins.rename(columns={'index': 'Jugador'})

top_20_victorias = df_most_wins.sort_values(by='Victorias', ascending=False).head(20)

chart_victorias = alt.Chart(top_20_victorias).mark_bar().encode(
    x=alt.X('Victorias:Q', title='Victorias'),
    y=alt.Y('Jugador:N', title='Jugador', sort='-x'),
    color=alt.Color('Jugador:N', legend=None),
    tooltip=['Jugador', 'Victorias']
).properties(
    title='Top 20 jugadores con más victorias',
    width=600
)

chart_victorias

alt.Chart(...)

### Rendimiento de cada jugador

In [205]:
jugador = "Rafael Nadal"
rend_jugador = result[result["jugador"] == jugador]

alt.Chart(rend_jugador).mark_line(
    point=alt.OverlayMarkDef(filled=False, fill="white")
).encode(
    x='año',
    y='Rendimiento',
    # color='symbol:N'
    tooltip=["año", "Rendimiento"]
).properties(
    title = 'Rendimiento por año de ' + jugador,
    width= 600,
    # height= 400
)

alt.Chart(...)

In [183]:
df_most_wins['Rendimiento'] = df_most_wins['Victorias'] / df_most_wins['Total_Partidos']
df_most_wins = df_most_wins[df_most_wins['Total_Partidos'] >= 100]
top_20_rendimiento = df_most_wins.sort_values(by='Rendimiento', ascending=False).head(20)

chart_rendimiento = alt.Chart(top_20_rendimiento).mark_bar().encode(
    x=alt.X('Rendimiento:Q', title='Rendimiento'),
    y=alt.Y('Jugador:N', title='Jugador', sort='-x'),
    color=alt.Color('Jugador:N',  legend=None),
    tooltip=['Jugador', 'Rendimiento']
).properties(
    title='Top 20 jugadores con mejor rendimiento (más de 100 partidos)',
    width=600
)

text = chart_rendimiento.mark_text(
    align='left',
    baseline='middle',
    dx=10,
    # color ='black'
).encode(
    text=alt.Text('Rendimiento:Q', format='.2f')
)

chart_rendimiento = (chart_rendimiento + text).interactive()

chart_rendimiento

alt.LayerChart(...)

In [184]:
victorias = defaultdict(int)

for file in file_list:
    df = pd.read_csv(file)

    for _, row in df.iterrows():
        winner = row['winner_name']
        loser = row['loser_name']

        victorias[winner] += 1

In [185]:
top_600_wins = [player for player,data in MostWinsDict.items() if data['Victorias'] >= 600]

In [186]:
inicio = []
final = []
partidos = []

for key , stats in H2HDict.items():
    try:
        key = sorted(key)
        # key es una tupla
        player1, player2 = key
        if player1 in top_600_wins and player2 in top_600_wins:
            inicio.append(player1)
            final.append(player2)
            partidos.append(stats['total_matches'])
    except ValueError as e:
       print(f"Error procesando {key}: {e}")


In [187]:
tenis_net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", notebook=True, cdn_resources='in_line')

edge_data = zip(inicio, final, partidos)
edge_data = set(edge_data)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    tenis_net.add_node(src, title=src)
    tenis_net.add_node(dst, title=dst)
    tenis_net.add_edge(src, dst, value=w)

In [188]:
G = nx.Graph()

for e in edge_data:
    G.add_edge(e[0], e[1])

# nx.draw(G, with_labels=True, node_color='blue', node_size=400, font_size=15, font_color='black', edge_color='gray')
degree_centrality = nx.degree_centrality(G)


In [189]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# La idea es teniendo la centralidad de grado, aplicar un gradiante de acuerdo al valor de centralida
# indicando que con un color mas oscuro una mayor centralidad
# Lo que quiero ver es si construyendo un grafo de solo tenistas con mas de 600 victorias
# Quien es el tenistas con mayor centralidad de grado, es decir, quien se enfrento a más tenistas con mas de 600 victorias


min_centrality = min(degree_centrality.values())
max_centrality = max(degree_centrality.values())

norm = mcolors.Normalize(vmin= min_centrality, vmax= max_centrality)
cmap = cm.get_cmap('Greens')

C:\Users\igroj\AppData\Local\Temp\ipykernel_23852\3762524255.py:14: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('Greens')


In [190]:
lista_rival = "H2H: \n"
for edge,data in H2HDict.items():
    p1 = edge[0]
    p2 = edge[1]
    if victorias[p1] >= 600 and victorias[p2] >= 600 and p1 == "Roger Federer":
        total = H2HDict[edge]["total_matches"]
        wins = H2HDict[edge]["wins_player1"]
        loses = total - wins
        lista_rival += " " + p2 + "total: " + str(total) + "victorias: " + str(wins) + "derrotas: " + str(loses)

lista_rival

'H2H: \n Andre Agassitotal: 11victorias: 8derrotas: 3 Yevgeny Kafelnikovtotal: 6victorias: 2derrotas: 4 Lleyton Hewitttotal: 27victorias: 18derrotas: 9 Goran Ivanisevictotal: 2victorias: 2derrotas: 0 Michael Changtotal: 5victorias: 4derrotas: 1 Andy Roddicktotal: 24victorias: 21derrotas: 3 Pete Samprastotal: 1victorias: 1derrotas: 0 David Ferrertotal: 17victorias: 17derrotas: 0 Rafael Nadaltotal: 41victorias: 17derrotas: 24 Tomas Berdychtotal: 26victorias: 20derrotas: 6 Andy Murraytotal: 25victorias: 14derrotas: 11 Richard Gasquettotal: 21victorias: 19derrotas: 2 Novak Djokovictotal: 51victorias: 23derrotas: 28'

In [191]:
for node in tenis_net.nodes:

    centrality = degree_centrality.get(node["id"], 0)
    color = mcolors.to_hex(cmap(norm(centrality)))
    node["label"] = node["id"]
    node["value"] = victorias[node['id']]
    node["color"] = color
    node["font"] = {"size":  victorias[node['id']]/35, "color": "white"}

    lista_rival = "H2H: \n"
    for edge,data in H2HDict.items():
        p1 = edge[0]
        p2 = edge[1]
        if victorias[p1] >= 600 and victorias[p2] >= 600 and p1 == node["id"]:
            total = H2HDict[edge]["total_matches"]
            wins = H2HDict[edge]["wins_player1"]
            loses = total - wins
            lista_rival += " " + p2 + ", total: " + str(total) + " victorias: " + str(wins) + " derrotas: " + str(loses) + "\n"

    node["title"] = node['id'] + ", Victorias:" + str(victorias[node["id"]]) + "\n" + lista_rival

tenis_net.show('tennis_top_players.html')

tennis_top_players.html
